In [9]:
import pandas as pd
import numpy as np

from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from gensim.models import LdaModel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv('../ready.csv').dropna()

In [5]:
# Есть ли пропуски:
print(df.isnull().sum())
# Есть ли дубликаты
print(df.duplicated().any())
print(len(df))

Unnamed: 0                         0
ucid                               0
message                            0
ACTION_ITEM_RESULT_PRODUCT_NAME    0
dtype: int64
False
42620


In [6]:
df.head()

,Unnamed: 0,ucid,message,ACTION_ITEM_RESULT_PRODUCT_NAME
0,0,"5,01000641613474E+017",добрый день это клиентский менеджер виктория с...,Бизнес-карта
1,1,"5,01001121614156E+017",добрый день звать клиентский менеджер сбербанк...,Зарплатные проекты
2,2,"5,01001811614178E+017",добрый день звать сбербанк блок сбер бизнес об...,Зарплатные проекты
3,3,"5,01002531614256E+017",добрый день клиентский менеджер удобно разгова...,Бизнес-карта
4,4,"5,01005041613457E+017",добрый день это сбербанк блок сбер бизнес клие...,Зарплатные проекты


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df.message, test_size=0.3, random_state=42)

In [22]:
from gensim import corpora

# Словарь
texts = [[word for word in doc.split()] for doc in X_train]
dictionary = corpora.Dictionary(texts)

# Корпус на основе матрицы подсчета слов
corpus = [dictionary.doc2bow(text) for text in texts]

In [28]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=16,
                     random_state=42,
                     update_every=1,
                     chunksize=500,
                     passes=15,
                     alpha='auto',
                     per_word_topics=True)

In [29]:
lda_model.save('../models/lda.model')

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

texts = [[word for word in doc.split()] for doc in X_test]
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.181423706689315

Coherence Score:  0.49316492604394724


In [31]:
# Вывод тем и соответствующих им ключевых слов
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}')

Topic: 0 
Words: 0.038*"пожалуйста" + 0.036*"линия" + 0.032*"информация" + 0.029*"мочь" + 0.022*"бизнес" + 0.021*"вопрос" + 0.020*"ваш" + 0.018*"организация" + 0.018*"клиент" + 0.017*"поддержка"
Topic: 1 
Words: 0.061*"зарплатный" + 0.053*"проект" + 0.043*"карта" + 0.024*"мочь" + 0.022*"переводить" + 0.022*"сотрудник" + 0.020*"перевод" + 0.019*"комиссия" + 0.017*"подключить" + 0.017*"это"
Topic: 2 
Words: 0.095*"терминал" + 0.091*"эквайринг" + 0.071*"торговый" + 0.035*"точка" + 0.029*"оборудование" + 0.025*"сбербанк" + 0.024*"установка" + 0.024*"оплата" + 0.022*"дополнительный" + 0.020*"установить"
Topic: 3 
Words: 0.020*"код" + 0.018*"это" + 0.017*"нужно" + 0.014*"нажимать" + 0.013*"зайти" + 0.012*"заявление" + 0.012*"далее" + 0.011*"бизнес" + 0.011*"нажать" + 0.011*"указать"
Topic: 4 
Words: 0.039*"вопрос" + 0.032*"услуга" + 0.027*"ваш" + 0.027*"время" + 0.025*"звонить" + 0.022*"добрый" + 0.020*"быть" + 0.020*"банк" + 0.019*"помочь" + 0.018*"дистанционно"
Topic: 5 
Words: 0.071*"карт

In [32]:
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.200039 -0.093866       1        1  16.131426
10    -0.104326 -0.171982       2        1  12.509339
6     -0.124143  0.029948       3        1  11.809272
11     0.269074 -0.174246       4        1  10.213470
1     -0.142983 -0.043196       5        1   8.038204
3     -0.024779  0.236437       6        1   5.800355
0      0.164230  0.129932       7        1   5.066813
5     -0.186087 -0.029596       8        1   4.700702
15     0.167412  0.140365       9        1   4.391124
7     -0.190848  0.034776      10        1   4.110047
14     0.058072 -0.019120      11        1   3.459125
9     -0.018937  0.164271      12        1   3.454925
13    -0.097853  0.060785      13        1   2.965122
12     0.156503 -0.056529      14        1   2.896933
8     -0.135940 -0.195388      15        1   2.628298
2      0.010567 -0.012591      16        1   1.824844, topic_info=            Term          Freq         Total Category  logprob  loglift
46         карта  99172.000000  99172.000000  Default  30.0000  30.0000
0         бизнес  70013.000000  70013.000000  Default  29.0000  29.0000
6         добрый  43793.000000  43793.000000  Default  28.0000  28.0000
256   зарплатный  17325.000000  17325.000000  Default  27.0000  27.0000
309       проект  15141.000000  15141.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
44        заявка    531.601993  11598.416751  Topic16  -4.7997   0.9209
46         карта    751.706084  99172.172326  Topic16  -4.4533  -0.8786
9           мочь    636.735595  61540.743624  Topic16  -4.6193  -0.5674
484            э    545.497072  17100.863118  Topic16  -4.7739   0.5585
31   возможность    475.573106  12036.105380  Topic16  -4.9111   0.7725

[1192 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2883      4  0.974205        expert
2661     12  0.992444          wifi
3065     16  0.812150           zoe
2227     14  0.971928  автоответчик
2637      1  0.989149     агентство
...     ...       ...           ...
774      14  0.003842   юридический
774      15  0.006083   юридический
774      16  0.018569   юридический
1071     10  0.960936         юрист
1071     14  0.037138         юрист

[3372 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 11, 7, 12, 2, 4, 1, 6, 16, 8, 15, 10, 14, 13, 9, 3])

In [64]:
# Функция для получения топика из текста
def get_topic(text):
    # tokens = preprocess_text(text)
    bow_vector = dictionary.doc2bow(text.split())
    topics_distribution = lda_model.get_document_topics(bow_vector)
    # Возвращаем номер топика с наибольшей вероятностью для данного документа
    return max(topics_distribution, key=lambda item: item[1])[0]

# Добавление столбца с номером топика для каждой строки
df['g_topics'] = df['message'].apply(get_topic)